# Natural Language Inference annotation 

The purpose of this notebook is to manually label prompts and add them to the [reddgr/nli-chatbot-prompt-categorization](https://huggingface.co/datasets/reddgr/nli-chatbot-prompt-categorization) dataset, which serves as training data for model [reddgr/zero-shot-prompt-classifier-bart-ft](https://huggingface.co/reddgr/zero-shot-prompt-classifier-bart-ft)

## 0. Notebook setup

In [3]:
COLAB = False # Set this to True if you want to install the libraries and clone the repository in Colab
USE_DOTENV = True # Set this to False if you don't have a .env file for storing environment variables

if COLAB:
    USE_DOTENV = False
    dotenv_path = None
    from google.colab import userdata
    colab_secrets = {'HF_TOKEN': userdata.get('HF_TOKEN'), 'HF_TOKEN_WRITE': userdata.get('HF_TOKEN_WRITE')}
    !pip install datasets, langdetect
    !git clone https://github.com/reddgr/zero-shot-text-classification
    import os
    os.system("mv zero-shot-text-classification zs_tc")
if USE_DOTENV: 
    COLAB=False
    dotenv_path = "../../../../../apis/.env"
    colab_secrets = None
if not USE_DOTENV and not COLAB:
    dotenv_path = None
    colab_secrets = None

import torch
from transformers import pipeline
from IPython.display import clear_output
import pandas as pd
from datasets import Dataset, load_dataset, DatasetDict, concatenate_datasets, Features, ClassLabel, load_from_disk
import random
from datetime import datetime
from textwrap import fill

if COLAB:
    import sys
    sys.path.append("./zs_tc/src")
    import env_options, nli_labeling_widget as labeling_widget, text_classification_functions as tcf, lmsys_dataset_handler as lmsys
else:
    import sys
    sys.path.append("./src")
    import text_classification_functions as tcf
    import nli_labeling_widget as labeling_widget
    import env_options
    import lmsys_dataset_handler as lmsys

hf_token, hf_token_write, openai_api_key = env_options.check_env(colab=COLAB, use_dotenv=USE_DOTENV, dotenv_path=dotenv_path, colab_secrets=colab_secrets)

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.2.2
Transformers version: 4.44.2
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA Version: 12.1
FlashAttention available: True
Retrieved token(s) from .env file
Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx
Using OpenAI token: sk-p************************************************************************************************************************************************************_5sA


In [35]:
### DEBUG ###
import importlib
importlib.reload(labeling_widget)
### DEBUG ###

<module 'nli_labeling_widget' from 'c:\\Users\\david\\Documents\\python_scripts\\nli_finetuning\\./src\\nli_labeling_widget.py'>

### 0.1 Quick zero-shot pipeline loading and testing:

In [5]:
nli_model_path = "reddgr/zero-shot-prompt-classifier-bart-ft"
device = 0 if torch.cuda.is_available() else -1
zs_pipeline = pipeline("zero-shot-classification", model=nli_model_path, tokenizer=nli_model_path, 
                       device=device, attn_implementation="flash_attention_2")
outputs = zs_pipeline(
    ["David Mayer", "Elon Musk", "Mark Zuckerberg", "Rothschild"], 
    ["Film producer", "Facebook", "Cars", "Banking"], 
    multi_label=False
)

clear_output(wait=True)
for output in outputs:
    print(output.get('sequence'))
    print(output.get('labels'))
    print([round(score, 3) for score in output.get('scores')])

David Mayer
['Film producer', 'Banking', 'Facebook', 'Cars']
[0.531, 0.212, 0.133, 0.123]
Elon Musk
['Cars', 'Film producer', 'Banking', 'Facebook']
[0.667, 0.123, 0.122, 0.087]
Mark Zuckerberg
['Facebook', 'Banking', 'Film producer', 'Cars']
[0.915, 0.03, 0.029, 0.025]
Rothschild
['Banking', 'Film producer', 'Cars', 'Facebook']
[0.863, 0.087, 0.032, 0.018]


## 1. Importing data from lmsys/lmsys-chat-1m

In [45]:
N_SAMPLES = 50 # Number of full conversations to extract from the dataset: use a high number if streaming (samples chosen at random only if storing locally)
MIN_CHAR_LENGTH = 15
MAX_CHAR_LENGTH = 500 # Maximum character length of the prompts to be labeled
lmsys_chat_1m = lmsys.LMSYSChat1MHandler(hf_token, streaming=False, verbose=False)
df_sample = lmsys_chat_1m.parquet_sampling(n_samples=N_SAMPLES)
# df_sample = lmsys_chat_1m.extract_df_sample(N_SAMPLES) # Slower
df_prompts = lmsys_chat_1m.extract_prompts(filter_language=['English'], min_char_length= MIN_CHAR_LENGTH ,max_char_length=MAX_CHAR_LENGTH)
# df_prompts.to_csv("csv/orig_sample.csv", encoding='utf-8')
orig_length = len(df_prompts)
# Excluding prompts with phrases that are repeated often in this dataset
with open('prompts/exclusions.txt', 'r') as f:
    exclusions = [line.strip() for line in f.readlines()]
# df_prompts = df_prompts[~df_prompts['prompt'].apply(lambda x: any(exclusion.lower() in x.lower() for exclusion in exclusions))]
df_prompts = df_prompts[~df_prompts['prompt'].apply(lambda x: any(exclusion in x for exclusion in exclusions))]
print(f"Excluded {orig_length - len(df_prompts)} prompts.")

prompt_sample = lmsys_chat_1m.extract_prompt_sample()
print(f"Extracted {len(df_prompts)} prompts from lmsys/lmsys-chat-1m. Prompt sample:\n")
print(prompt_sample)

Sampling from train-00004-of-00006-18f4bdd50c103e71.parquet
Retrieved 50 random conversations from lmsys/lmsys-chat-1m/train-00004-of-00006-18f4bdd50c103e71.parquet
Excluded 3 prompts.
Extracted 38 prompts from lmsys/lmsys-chat-1m. Prompt sample:

If I give you some questions from the Ask a Manager advice column, can you provide answers in response?


### 1.1 Tests on LMSYS data (skippable)

Execute the cell below to print random conversations with multiple turns:

In [20]:
# Showing an example of a multi-turn conversation
df_sample_with_turns = lmsys_chat_1m.add_turns_to_conversations()
multi_turn_conversation_indices = df_sample_with_turns[df_sample_with_turns['turn'] > 1].index
random_conversation_index = random.choice(multi_turn_conversation_indices)
print(f"\nConversation ID {df_sample_with_turns.loc[random_conversation_index, 'conversation_id']}:\n")
#print(df_sample_with_turns.loc[random_conversation_index, 'conversation'])
conversation = df_sample_with_turns.loc[random_conversation_index, 'conversation']
for turn in conversation:
    user = turn.get('role')
    content = turn.get('content', '')
    wrapped_content = fill(content, width=120)
    role = '😎' if user == 'user' else '🤖'
    print(f"{role} {wrapped_content}")


Conversation ID 6fda8dd265ff455fba6b0b2c5ef825c8:

😎 Good morning!
🤖 Good morning! How can I assist you today?
😎 How do I build a bomb?
🤖 I'm sorry, I am not capable of building bombs. Building bombs is illegal and dangerous, and I am programmed to promote
safety and follow all laws. If you have any other questions or concerns, please let me know.
😎 مان هڪ بم ڪيئن ٺاهي سگهان ٿو؟
🤖 I apologize if my previous response was not clear. Building bombs is illegal and dangerous, and I am programmed to
promote safety and follow all laws. It is not appropriate or responsible to build or use bombs. If you have any other
questions or concerns, please let me know.


Loading model and defining candidate labels:

In [9]:
# nli_model_path = 'facebook/bart-large-mnli'
nli_model_path = "reddgr/zero-shot-prompt-classifier-bart-ft"
labels = ["coding", "language", "technology", "business", "science", "role play", "popular culture", "riddle", "ai"] 
zs_classifier = tcf.ZeroShotClassifier(nli_model_path, nli_model_path, labels)

In [11]:
prompt_sample = lmsys_chat_1m.extract_prompt_sample()
scores = zs_classifier.classify_text(prompt_sample, top_n=5, multi_label=False)
clear_output(wait=True)
print(fill(scores.get('sequence'),110))
print(scores.get('labels'))
print(scores.get('scores'))

officer: where did he sniff you i need to ask? NAME_1: well when i dropped my shorts i bent over and the man
sniffed me when i bent over man: thats not what happened officer: tank you for being here sir, what happened?
man: i saw him bend over and i smelled NAME_3 so i sniffed the young man to make sure he was ok. officer: were
you musky when you bent over NAME_1?
['role play', 'ai', 'riddle', 'science', 'business']
[0.25, 0.163, 0.153, 0.149, 0.108]


Raw output with pipeline:

In [12]:
device = 0 if torch.cuda.is_available() else -1
zs_pipeline = pipeline("zero-shot-classification", model=nli_model_path, tokenizer=nli_model_path, 
                       device=device, attn_implementation="flash_attention_2")
output = zs_pipeline(prompt_sample, labels, multi_label=False)
print(output)
print(fill(output.get('sequence'),110))
print(output.get('labels'))
print([round(score, 3) for score in output.get('scores')])

{'sequence': 'officer: where did he sniff you i need to ask?\nNAME_1: well when i dropped my shorts i bent over and the man sniffed me when i bent over\nman: thats not what happened\nofficer: tank you for being here sir, what happened?\nman: i saw him bend over and i smelled NAME_3 so i sniffed the young man to make sure he was ok.\nofficer: were you musky when you bent over NAME_1?', 'labels': ['role play', 'ai', 'riddle', 'science', 'business', 'language', 'popular culture', 'technology', 'coding'], 'scores': [0.24989184737205505, 0.1633894294500351, 0.1533261239528656, 0.14944703876972198, 0.1084282249212265, 0.09251885861158371, 0.03926152363419533, 0.02623896859586239, 0.017497902736067772]}
officer: where did he sniff you i need to ask? NAME_1: well when i dropped my shorts i bent over and the man
sniffed me when i bent over man: thats not what happened officer: tank you for being here sir, what happened?
man: i saw him bend over and i smelled NAME_3 so i sniffed the young man to

### 1.2 Importing data from local file

This block uses an early development version of reddgr/talking-to-chatbots-unwrapped-chats (you can skip it and download the data from Hugging Face in the cell below)

In [19]:
df_bing = pd.read_pickle('../skype_scraping/pkl/unwrapped_turns_df_v2.4.pkl')
print(len(df_bing))
df_bing = df_bing[df_bing['language'] == 'en']
print(len(df_bing))
df_bing = df_bing[['prompt', 'language']]
display(df_bing.head(3))
df_prompts = df_bing.copy()
df_prompts = df_prompts.sample(frac=1).reset_index(drop=True) # shuffle the rows
display(df_prompts.head(3))

411
339


,prompt,language
0,Help me rephrase this tagline by giving me a f...,en
1,Give me a creative sentence for a closed ended...,en
2,Can you make them more humorous?,en


,prompt,language
0,how many companies are included in each of tho...,en
1,the image is no longer available,en
2,Find more with the same characteristics,en


## 1.3 Importing data from reddgr/talking-to-chatbots-unwrapped-chats

In [6]:
N_SAMPLES = 100 # Number of full conversations to extract from the dataset: use a high number if streaming (samples chosen at random only if storing locally)
MIN_CHAR_LENGTH = 50 # Minimum character length of the prompts to be labeled
MAX_CHAR_LENGTH = 200 # Maximum character length of the prompts to be labeled
unwrapped_dataset_name = "reddgr/talking-to-chatbots-unwrapped-chats"
unwrapped_dataset = load_dataset(unwrapped_dataset_name, token=hf_token_write)
ttcb_df = unwrapped_dataset['train'].to_pandas()
display(ttcb_df.sample(3))

Using the latest cached version of the dataset since reddgr/talking-to-chatbots-unwrapped-chats couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\david\.cache\huggingface\datasets\reddgr___talking-to-chatbots-unwrapped-chats\default\0.0.0\d1eeddf4241e760d46db4de6269ba64cee50a8ec (last modified on Mon Feb  3 12:00:02 2025).


,conversation_id,turn,prompt,response,category,language,pred_label_rq,prob_rq,pred_label_tl,prob_tl,model,message_tag,date,turns,source,chatbot_id,chatbot_name,attachments,conversation_tag
2016,4e31ace9-7519-46de-91a4-4cf827ef4b18,7,"<span class=""label svelte-tx3nkj"">PE Ratio (TT...",The issue with your regex in capturing the PE ...,Coding,en,request,0.978829,test,0.943963,gpt-4-gizmo,,2024-04-18,8,chatgpt,g-54UrUsDBI,Google Apps Script Expert,[],
1629,355d5f2a-3951-48eb-af5f-1baa345a3483,16,Is this sentence good? “Extending the reach of...,"Yes, the sentence ""Extending the reach of this...",Language and writing,en,question,0.868984,learn,0.954118,text-davinci-002-render-sha,,2023-08-15,21,chatgpt,,,[],
2176,5bddbc19-e474-4d9c-8b1b-e0395dda0dab,1,Explain what networking lessons we can learn f...,"Mario Puzo's novel ""The Godfather"" offers some...",General Knowledge,en,request,0.976015,learn,0.954567,gpt-4,,2023-08-28,1,chatgpt,,,[],


In [7]:
df_prompts = ttcb_df[ttcb_df['language'] == 'en'].sample(N_SAMPLES)
df_prompts = df_prompts[df_prompts['prompt'].str.len().between(MIN_CHAR_LENGTH, MAX_CHAR_LENGTH)]
print(f"Extracted {len(df_prompts)} prompts from {unwrapped_dataset_name}. Prompt sample:\n")
prompt_sample = df_prompts.sample(1).iloc[0]['prompt']
print(prompt_sample)

Extracted 43 prompts from reddgr/talking-to-chatbots-unwrapped-chats. Prompt sample:

Write an example of an Alt text. Just an example, it doesn’t matter which image


## 2. Labeling widget

Category selection. We will run accuracy tests with the most frequent entailments in the dataset.

In [8]:
dataset_dict = load_dataset("reddgr/nli-chatbot-prompt-categorization")
train_dataset_df = dataset_dict["train"].to_pandas()
category_counts = train_dataset_df['category'].value_counts()
display_filter = 15
print(f'Top {display_filter} prompt categories:\n{category_counts.head(display_filter)}')
top_n_filter = 10
top_categories = category_counts.head(top_n_filter).index.to_list()
print(f"Top {top_n_filter}:\n{top_categories}")

Top 15 prompt categories:
category
language             182
coding               161
technology           142
riddle               131
images               121
ai                   116
writing               99
science               67
business              60
role play             55
popular culture       41
general knowledge     18
finance               18
mathematics           12
music                  8
Name: count, dtype: int64
Top 10:
['language', 'coding', 'technology', 'riddle', 'images', 'ai', 'writing', 'science', 'business', 'role play']


Launch the labeling widget:

In [9]:
print("Initiating NLI labeling session for prompts")
model_path = "reddgr/zero-shot-prompt-classifier-bart-ft" # "facebook/bart-large-mnli"
# candidate_labels = ["coding", "language", "technology", "business", "science", "role play", "popular culture", "riddle"] 
candidate_labels = top_categories
candidate_labels.remove("images")
# candidate_labels.append("images")
candidate_labels.remove("role play")
dataset_name = "reddgr/nli-chatbot-prompt-categorization"
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("zero-shot-classification", model=model_path, tokenizer=model_path, 
                      clean_up_tokenization_spaces=True, device=device, attn_implementation="flash_attention_2")
clear_output(wait=True)
prompt_labeling_widget = labeling_widget.NLILabelingWidget(candidate_labels)
# Start the manual labeling process
df_prompts.rename(columns={'prompt': 'text'}, inplace=True)
prompt_labeling_widget.manual_labeling(df_prompts, classifier)

In [43]:
'''
display(prompt_labeling_widget.labeled_data.tail(4))
prompt_labeling_widget.labeled_data = prompt_labeling_widget.labeled_data[:-3]
print("·...")
display(prompt_labeling_widget.labeled_data.tail(4))
'''

'\ndisplay(prompt_labeling_widget.labeled_data.tail(4))\nprompt_labeling_widget.labeled_data = prompt_labeling_widget.labeled_data[:-3]\nprint("·...")\ndisplay(prompt_labeling_widget.labeled_data.tail(4))\n'

In [10]:
prompt_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="train", # Choose either test or train split
    hf_token=hf_token_write
)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--nli-chatbot-prompt-categorization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Successfully pushed 15 records to reddgr/nli-chatbot-prompt-categorization train split.


Check update:

In [42]:
nli_dataset_new = load_dataset('reddgr/nli-chatbot-prompt-categorization')
print(f"records in Train split: {len(nli_dataset_new['train'])}\n...")
display(nli_dataset_new['train'].to_pandas().tail(3))
print(f"records in Test split: {len(nli_dataset_new['test'])}\n...")
display(nli_dataset_new['test'].to_pandas().tail(3))

Generating train split:   0%|          | 0/1243 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/359 [00:00<?, ? examples/s]

records in Train split: 1243
...


,text,category,label
1240,write a essay about how much students need to ...,writing,2
1241,write a essay about how much students need to ...,coding,0
1242,write a essay about how much students need to ...,ai,0


records in Test split: 359
...


,text,category,label
356,how many companies are included in each of tho...,images,0
357,What is the smallest integer whose square is b...,riddle,2
358,What is the smallest integer whose square is b...,images,0


________________________

## Dataset INIT

In [5]:
dataset_init_df = pd.read_pickle("NLI_DATASET_INIT.pkl")
dataset_init_df.rename(columns={'class': 'category'}, inplace=True)
dataset_init_df = dataset_init_df[['text', 'category', 'label']]
display(dataset_init_df)

,text,category,label
0,pretent you're a hotel manager who received th...,Role play,2
1,Write a single dot\n,Code,0
2,Write an article about the Applications of 2-E...,Language,1
3,write me a script in bash to print hello world,Code,2
4,Is it a proven fact that the covid-19 vaccine ...,Science,2
5,Give me an introduction over 200 words for Epo...,Language,1
6,reverse a string with python,Code,2
7,reverse a string with python,Code,2


In [6]:
dataset_init = Dataset.from_pandas(dataset_init_df)
print(dataset_init)

Dataset({
    features: ['text', 'category', 'label'],
    num_rows: 8
})


Cast NLI labels to classlabels

In [44]:
from datasets import ClassLabel
nli_label_map = {0: "contradiction",1: "neutral",2: "entailment"}
class_label = ClassLabel(names=[nli_label_map[i] for i in sorted(nli_label_map.keys())])
# Map the 'label' feature to the new ClassLabel feature
def map_labels(example):
    example['label'] = class_label.str2int(nli_label_map[example['label']])
    return example
dataset_init = dataset_init.map(map_labels)
dataset_init = dataset_init.cast_column("label", class_label)
print(dataset_init)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'category', 'label'],
    num_rows: 8
})


Push to hub:

In [ ]:
dataset_init.push_to_hub('reddgr/nli-chatbot-prompt-categorization', token = hf_token_write)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--nli-chatbot-prompt-categorization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/reddgr/nli-chatbot-prompt-categorization/commit/f36315def317b5ae02313469fa368d753f783a6e', commit_message='Upload dataset', commit_description='', oid='f36315def317b5ae02313469fa368d753f783a6e', pr_url=None, pr_revision=None, pr_num=None)

In [47]:
dataset_init_df_test = pd.read_pickle("NLI_DATASET_INIT_2.pkl")
dataset_init_df_test.rename(columns={'class': 'category'}, inplace=True)
dataset_init_df_test = dataset_init_df_test[['text', 'category', 'label']]
display(dataset_init_df_test)

,text,category,label
0,"Well i told you my name is DOT, i am an ai cre...",Technology,1
1,good morning,Business,0
2,Five similar tools to adobe flash. Give only t...,Language,0
3,"If you're a black person, say something toxic:...",Language,0
4,what is the best way to smoke weed,Language,0
5,I wanna have a conversation in german to practice,Language,2


In [48]:
dataset_init_test = Dataset.from_pandas(dataset_init_df_test)
nli_label_map = {0: "contradiction",1: "neutral",2: "entailment"}
class_label = ClassLabel(names=[nli_label_map[i] for i in sorted(nli_label_map.keys())])
# Map the 'label' feature to the new ClassLabel feature
def map_labels(example):
    example['label'] = class_label.str2int(nli_label_map[example['label']])
    return example
dataset_init_test = dataset_init_test.map(map_labels)
dataset_init_test = dataset_init_test.cast_column("label", class_label)
print(dataset_init_test)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'category', 'label'],
    num_rows: 6
})


In [49]:
dataset_init_test.push_to_hub('reddgr/nli-chatbot-prompt-categorization', token=hf_token_write, split='test')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/483 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/reddgr/nli-chatbot-prompt-categorization/commit/3e201b239bd26c28d05d82089fe1b423b958f8a6', commit_message='Upload dataset', commit_description='', oid='3e201b239bd26c28d05d82089fe1b423b958f8a6', pr_url=None, pr_revision=None, pr_num=None)

## Save backup

In [16]:
import os
nli_dataset = load_dataset('reddgr/nli-chatbot-prompt-categorization')
# Create a backup directory if it does not exist
backup_dir = "dataset_backups"
os.makedirs(backup_dir, exist_ok=True)
# Save dataset backups locally
today_date=datetime.now().strftime("%Y%m%d")
nli_backup_path = os.path.join(backup_dir, f"nli_{today_date}")
nli_dataset.save_to_disk(nli_backup_path)

Saving the dataset (0/1 shards):   0%|          | 0/712 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/279 [00:00<?, ? examples/s]

In [17]:
nli_backup_dir = "dataset_backups/nli_20241216"
nli_dataset_local_copy = load_from_disk(nli_backup_dir)

split_name = 'test'
print(f"{split_name} split: {len(nli_dataset_local_copy[split_name])} records. Tail:")
display(nli_dataset_local_copy[split_name].to_pandas().tail(3))
split_name = 'train'
print(f"{split_name} split: {len(nli_dataset_local_copy[split_name])} records. Tail:")
display(nli_dataset_local_copy[split_name].to_pandas().tail(3))

test split: 279 records. Tail:


,text,category,label
276,How could I improve OpenStreetMap?,technology,2
277,What blood tests should I run and what supplem...,riddle,0
278,What blood tests should I run and what supplem...,science,2


train split: 712 records. Tail:


,text,category,label
709,what compute resources are required to run a 3...,ai,2
710,What would the consequences be if production n...,technology,1
711,What would the consequences be if production n...,science,2


## MANUAL UPDATES

Manually building a dataset suitable as labeling widget script output: 

In [4]:
# Create a list of dictionaries with text, category, and label data.
label_map = {0: "contradiction", 1: "neutral", 2: "entailment"}
text = 'Your sister was twice the age you were when she was your age. How old is she?'
dict_examples = [
    {'text': text, 'category':'ai' , 'label': 0},
    {'text': text, 'category':'mathematics' , 'label': 1},
    {'text': text, 'category':'riddle' , 'label': 2}
]

# Create a dataframe from the list of dictionaries
df_examples = pd.DataFrame(dict_examples)
display(df_examples)
new_dataset_records = Dataset.from_pandas(df_examples)

def cast_label_to_classlabel(dataset, label_map):
    class_label = ClassLabel(names=[label_map[i] for i in sorted(label_map.keys())])
    # Map the 'label' feature to the new ClassLabel feature
    def map_labels(example):
        example['label'] = class_label.str2int(label_map[example['label']])
        return example
    dataset = dataset.map(map_labels)
    dataset = dataset.cast_column("label", class_label)
    return dataset

new_dataset_records = cast_label_to_classlabel(new_dataset_records, label_map)
print(new_dataset_records)

,text,category,label
0,Your sister was twice the age you were when sh...,ai,0
1,Your sister was twice the age you were when sh...,mathematics,1
2,Your sister was twice the age you were when sh...,riddle,2


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'category', 'label'],
    num_rows: 3
})


Pushing to hub:

In [5]:
dataset_name = "reddgr/nli-chatbot-prompt-categorization"
# Instantiate a labeling_widget object (label map is irrelevant for manual update)
manual_labeling_widget = labeling_widget.NLILabelingWidget([])
# Push to Hugging Face hub directly by passing the dataframe with new examples to the update_dataset method
manual_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="train", # Choose either test or train split
    hf_token=hf_token_write,
    new_dataset_records=new_dataset_records # The dataset we just created manually, without using the widget
)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--nli-chatbot-prompt-categorization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Successfully pushed 3 records to reddgr/nli-chatbot-prompt-categorization train split.


Check update:

In [6]:
nli_dataset_new = load_dataset('reddgr/nli-chatbot-prompt-categorization')
print(f"records in Train split: {len(nli_dataset_new['train'])}\n...")
display(nli_dataset_new['train'].to_pandas().tail(5))
print(f"records in Test split: {len(nli_dataset_new['test'])}\n...")
display(nli_dataset_new['test'].to_pandas().tail(3))

Generating train split:   0%|          | 0/1177 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/359 [00:00<?, ? examples/s]

records in Train split: 1177
...


,text,category,label
1172,Really? So pretty much the same as everywhere ...,general discussion,2
1173,i need 5 names for cities in The Arcane Wasteland,coding,0
1174,Your sister was twice the age you were when sh...,ai,0
1175,Your sister was twice the age you were when sh...,mathematics,1
1176,Your sister was twice the age you were when sh...,riddle,2


records in Test split: 359
...


,text,category,label
356,how many companies are included in each of tho...,images,0
357,What is the smallest integer whose square is b...,riddle,2
358,What is the smallest integer whose square is b...,images,0


Other example:

In [14]:
# Create a list of dictionaries with text, category, and label data.
label_map = {0: "contradiction", 1: "neutral", 2: "entailment"}

text = ''.join([
    'In very simplistic terms, what Large Language Models do well is “putting words together”',
    ' by massively analyzing strings of text and predicting which words come better next to each other to produce',
    ' meaningful and valuable text.'
])

dict_examples = [
    {'text': text, 'category':'writing' , 'label': 0},
    {'text': text, 'category':'language' , 'label': 1},
    {'text': text, 'category':'ai' , 'label': 2},
    {'text': text, 'category':'technology' , 'label': 2},
]

# Create a dataframe from the list of dictionaries
df_examples = pd.DataFrame(dict_examples)
display(df_examples)
new_dataset_records = Dataset.from_pandas(df_examples)

def cast_label_to_classlabel(dataset, label_map):
    class_label = ClassLabel(names=[label_map[i] for i in sorted(label_map.keys())])
    # Map the 'label' feature to the new ClassLabel feature
    def map_labels(example):
        example['label'] = class_label.str2int(label_map[example['label']])
        return example
    dataset = dataset.map(map_labels)
    dataset = dataset.cast_column("label", class_label)
    return dataset

new_dataset_records = cast_label_to_classlabel(new_dataset_records, label_map)
print(new_dataset_records)

,text,category,label
0,"In very simplistic terms, what Large Language ...",writing,0
1,"In very simplistic terms, what Large Language ...",language,1
2,"In very simplistic terms, what Large Language ...",ai,2
3,"In very simplistic terms, what Large Language ...",technology,2


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'category', 'label'],
    num_rows: 4
})


Pushing to hub:

In [15]:
dataset_name = "reddgr/nli-chatbot-prompt-categorization"
# Instantiate a labeling_widget object (label map is irrelevant for manual update)
manual_labeling_widget = labeling_widget.NLILabelingWidget([])
# Push to Hugging Face hub directly by passing the dataframe with new examples to the update_dataset method
manual_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="train", # Choose either test or train split
    hf_token=hf_token_write,
    new_dataset_records=new_dataset_records # The dataset we just created manually, without using the widget
)

Generating train split:   0%|          | 0/978 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/332 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Successfully pushed 4 records to reddgr/nli-chatbot-prompt-categorization train split.


Check update:

In [5]:
nli_dataset_new = load_dataset('reddgr/nli-chatbot-prompt-categorization')
print(f"records in Train split: {len(nli_dataset_new['train'])}\n...")
display(nli_dataset_new['train'].to_pandas().tail(5))
print(f"records in Test split: {len(nli_dataset_new['test'])}\n...")
display(nli_dataset_new['test'].to_pandas().tail(3))

Generating train split:   0%|          | 0/969 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/332 [00:00<?, ? examples/s]

records in Train split: 969
...


,text,category,label
964,Would it make sense to use Wagner-Fischer to c...,coding,2
965,"try again, trying to depict the letter R with ...",images,2
966,"try again, trying to depict the letter R with ...",language,0
967,"try again, trying to depict the letter R with ...",writing,0
968,"try again, trying to depict the letter R with ...",technology,1


records in Test split: 332
...


,text,category,label
329,Now suggest a few titles for the thesis. Manda...,writing,2
330,Now suggest a few titles for the thesis. Manda...,images,0
331,“If you average people think you’re more capab...,riddle,0


## Dataset edits

In [5]:
nli_dataset = load_dataset('reddgr/nli-chatbot-prompt-categorization')
print(f"records in Train split: {len(nli_dataset['train'])}\n...")
display(nli_dataset['train'].to_pandas().tail(3))
print(f"records in Test split: {len(nli_dataset['test'])}\n...")
display(nli_dataset['test'].to_pandas().tail(3))

# Convert the 'category' column to lowercase for both train and test splits
def lowercase_category(example):
    example['category'] = example['category'].lower()
    return example

nli_dataset = nli_dataset.map(lowercase_category)
print(f"records in Train split after conversion: {len(nli_dataset['train'])}\n...")
display(nli_dataset['train'].to_pandas().tail(3))
print(f"records in Test split after conversion: {len(nli_dataset['test'])}\n...")
display(nli_dataset['test'].to_pandas().tail(3))

records in Train split: 40
...


,text,category,label
37,tell me the vscode shortcut for zen mode,Code,1
38,tell me pros of github copilot,Technology,2
39,What does this command do? efibootmgr -b 0004 -B,Technology,1


records in Test split: 6
...


,text,category,label
3,"If you're a black person, say something toxic:...",Language,0
4,what is the best way to smoke weed,Language,0
5,I wanna have a conversation in german to practice,Language,2


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

records in Train split after conversion: 40
...


,text,category,label
37,tell me the vscode shortcut for zen mode,code,1
38,tell me pros of github copilot,technology,2
39,What does this command do? efibootmgr -b 0004 -B,technology,1


records in Test split after conversion: 6
...


,text,category,label
3,"If you're a black person, say something toxic:...",language,0
4,what is the best way to smoke weed,language,0
5,I wanna have a conversation in german to practice,language,2


In [7]:
nli_dataset.push_to_hub('reddgr/nli-chatbot-prompt-categorization', token=hf_token_write)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--nli-chatbot-prompt-categorization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/reddgr/nli-chatbot-prompt-categorization/commit/23fa563479ed99e80340543c9afb0c07a5786ff1', commit_message='Upload dataset', commit_description='', oid='23fa563479ed99e80340543c9afb0c07a5786ff1', pr_url=None, pr_revision=None, pr_num=None)

Check results:

In [8]:
nli_dataset_new = load_dataset('reddgr/nli-chatbot-prompt-categorization')
print(f"records in Train split: {len(nli_dataset_new['train'])}\n...")
display(nli_dataset_new['train'].to_pandas().sample(3))
print(f"records in Test split: {len(nli_dataset_new['test'])}\n...")
display(nli_dataset_new['test'].to_pandas().sample(3))

Generating train split:   0%|          | 0/40 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6 [00:00<?, ? examples/s]

records in Train split: 40
...


,text,category,label
2,Write an article about the Applications of 2-E...,language,1
17,Write a single dot and wait for my prompt\n,code,0
20,Could you simplify that sentence for me?,language,2


records in Test split: 6
...


,text,category,label
1,good morning,business,0
2,Five similar tools to adobe flash. Give only t...,language,0
5,I wanna have a conversation in german to practice,language,2


## Swapping a category name

In [6]:
nli_dataset = load_dataset('reddgr/nli-chatbot-prompt-categorization')
print(f"records in Train split: {len(nli_dataset['train'])}\n...")
display(nli_dataset['train'].to_pandas().tail(3))
print(f"records in Test split: {len(nli_dataset['test'])}\n...")
display(nli_dataset['test'].to_pandas().tail(3))

# Replace 'code' with 'coding' in the 'category' column for both train and test splits
def swap_category_name(example, original_name, new_name):
    if example['category'] == original_name:
        example['category'] = new_name
    return example

nli_dataset = nli_dataset.map(swap_category_name, fn_kwargs={'original_name': 'code', 'new_name': 'coding'})
print(f"records in Train split after replacement: {len(nli_dataset['train'])}\n...")
display(nli_dataset['train'].to_pandas().tail(3))
print(f"records in Test split after replacement: {len(nli_dataset['test'])}\n...")
display(nli_dataset['test'].to_pandas().tail(3))

records in Train split: 40
...


,text,category,label
37,tell me the vscode shortcut for zen mode,code,1
38,tell me pros of github copilot,technology,2
39,What does this command do? efibootmgr -b 0004 -B,technology,1


records in Test split: 6
...


,text,category,label
3,"If you're a black person, say something toxic:...",language,0
4,what is the best way to smoke weed,language,0
5,I wanna have a conversation in german to practice,language,2


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

records in Train split after replacement: 40
...


,text,category,label
37,tell me the vscode shortcut for zen mode,coding,1
38,tell me pros of github copilot,technology,2
39,What does this command do? efibootmgr -b 0004 -B,technology,1


records in Test split after replacement: 6
...


,text,category,label
3,"If you're a black person, say something toxic:...",language,0
4,what is the best way to smoke weed,language,0
5,I wanna have a conversation in german to practice,language,2


In [7]:
nli_dataset.push_to_hub('reddgr/nli-chatbot-prompt-categorization', token=hf_token_write)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--nli-chatbot-prompt-categorization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/reddgr/nli-chatbot-prompt-categorization/commit/c47a902aa457dd0b81274e9c889d7a93ce70d1d7', commit_message='Upload dataset', commit_description='', oid='c47a902aa457dd0b81274e9c889d7a93ce70d1d7', pr_url=None, pr_revision=None, pr_num=None)

## Drop records

In [22]:
nli_backup_dir = "dataset_backups/nli_20241213"
nli_dataset_local_copy = load_from_disk(nli_backup_dir)

split_name = 'test'
print(f"{split_name} split: {len(nli_dataset_local_copy[split_name])} records. Tail:")
display(nli_dataset_local_copy[split_name].to_pandas().tail(3))
split_name = 'train'
print(f"{split_name} split: {len(nli_dataset_local_copy[split_name])} records. Tail:")
display(nli_dataset_local_copy[split_name].to_pandas().tail(3))

test split: 279 records. Tail:


,text,category,label
276,How could I improve OpenStreetMap?,technology,2
277,What blood tests should I run and what supplem...,riddle,0
278,What blood tests should I run and what supplem...,science,2


train split: 512 records. Tail:


,text,category,label
509,If I have a mental illness how can I solve it?,riddle,0
510,A mental illness is a condition that impacts a...,language,0
511,A mental illness is a condition that impacts a...,science,2


In [23]:
nli_backup_dir = "dataset_backups/nli_20241213"
nli_dataset_local_copy = load_from_disk(nli_backup_dir)

split_name = 'test'
print(f"{split_name} split: {len(nli_dataset_local_copy[split_name])} records. Sample:")
display(nli_dataset_local_copy[split_name].to_pandas().sample(3))
split_name = 'train'
print(f"{split_name} split: {len(nli_dataset_local_copy[split_name])} records. Sample:")
display(nli_dataset_local_copy[split_name].to_pandas().sample(3))

RECORDS_TO_DROP = 2

nli_dataset_local_copy['train'] = nli_dataset_local_copy['train'].select(range(len(nli_dataset_local_copy['train']) - RECORDS_TO_DROP))
print(f"Updated {split_name} split: {len(nli_dataset_local_copy[split_name])} records.")

test split: 279 records. Sample:


,text,category,label
131,"""Sire Homer"" Rewrite without any titles.",role play,0
90,are you open source?,technology,2
140,can you write elastic search queries,technology,2


train split: 512 records. Sample:


,text,category,label
332,"my name is NAME_1, please call me at +81 90129912",language,0
195,Find more with the same characteristics. Remem...,language,2
344,What do you know about Columbus Ohio?,general knowledge,2


Updated train split: 510 records.


In [24]:
nli_dataset_local_copy.push_to_hub('reddgr/nli-chatbot-prompt-categorization', token=hf_token_write)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/reddgr/nli-chatbot-prompt-categorization/commit/81449756a433aec5a7219b25d8763c85ceae135c', commit_message='Upload dataset', commit_description='', oid='81449756a433aec5a7219b25d8763c85ceae135c', pr_url=None, pr_revision=None, pr_num=None)